In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook
from sklearn import svm, metrics, datasets
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split
from skimage.io import imread
from skimage.transform import resize
import skimage

In [2]:

def load_image_files(main_path,img_dim=(64, 64)):
    """
    This function Loads image files which works like scikit-learn.This image file has
    categories as subfolder names such as c1,c2,c3...c9
    
    Parameters
    ----------
    main_path : string Path which contains one subfolder per category.
    img_dim : Size of the tuple to which the raw_input are adjusted.
        
    Returns
    -------
    Bunch
    """
    imgs_direc = Path(main_path)
    img_sub_folder = [directory for directory in imgs_direc.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in img_sub_folder]

    class_data = "A image classification dataset"
    raw_input = []
    flat_data = []
    target = []
    for i, direc in enumerate(img_sub_folder):
        for file in direc.iterdir():
            img = skimage.io.imread(file)
            img_resized = resize(img, img_dim, anti_aliasing=True, mode='reflect')
            flat_data.append(img_resized.flatten()) 
            raw_input.append(img_resized)
            target.append(i)
    flat_data = np.array(flat_data)
    target = np.array(target)
    raw_input = np.array(raw_input)

    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 raw_input=raw_input,
                 class_data=class_data)

In [3]:
image_dataset = load_image_files("/Users/patil1695/Downloads/data/imgs/train/")

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    image_dataset.data, image_dataset.target, test_size=0.3,random_state=101)

In [5]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
svc = svm.SVC()
clf = GridSearchCV(svc, param_grid,refit=True)
clf = clf.fit(X_train, y_train)

/Users/patil1695/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [ ]:
clf = clf.fit(X_train, y_train)
grid_scores = clf.grid_scores_


/Users/patil1695/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print("Classification report for - \n{}:\n{}\n".format(
    clf, metrics.classification_report(y_test, y_pred)))

In [ ]:
print("Confusion Matrix for - \n{}:\n{}\n".format(
    clf, metrics.confusion_matrix(y_test, y_pred, labels=None, sample_weight=None)))

In [ ]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(clf, open(filename, 'wb'))

loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

In [ ]:
print(clf.best_score_)